In [1]:
import timeit
start = timeit.default_timer()

import os
os.environ['PYARROW_IGNORE_TIMEZONE'] = '1'
import sys
print(sys.version)

3.10.9 (main, Mar  1 2023, 18:23:06) [GCC 11.2.0]


In [2]:
import folium
from folium.plugins import MarkerCluster
import geohash_tools as gh
import numpy as np
import pyarrow as pa
import pyspark as ps

In [3]:
sp = spark.read.load('hdfs://orion11:14001/nam/2019/01/05/*',
                     format='csv',
                     sep='\t',
                     inferSchema='true',
                     header='true')

schema = sp.schema

sp = spark.read.schema(schema).load('hdfs://orion11:14001/nam/201*/*/*/*',
                     format='csv',
                     sep='\t',
                     header='true')

23/05/22 02:35:18 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/05/22 02:35:33 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/05/22 02:35:48 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources


In [4]:
sp.printSchema()

root
 |-- 1_time: long (nullable = true)
 |-- 2_lat: double (nullable = true)
 |-- 3_lon: double (nullable = true)
 |-- albedo_surface: double (nullable = true)
 |-- precipitable_water_entire_atmosphere_single_layer: double (nullable = true)
 |-- pressure_maximum_wind: double (nullable = true)
 |-- pressure_surface: double (nullable = true)
 |-- pressure_tropopause: double (nullable = true)
 |-- relative_humidity_zerodegc_isotherm: double (nullable = true)
 |-- snow_depth_surface: double (nullable = true)
 |-- temperature_surface: double (nullable = true)
 |-- temperature_tropopause: double (nullable = true)
 |-- total_cloud_cover_entire_atmosphere_single_layer: double (nullable = true)
 |-- total_precipitation_surface_3_hour_accumulation: double (nullable = true)
 |-- vegetation_surface: double (nullable = true)
 |-- visibility_surface: double (nullable = true)
 |-- wilting_point_surface: double (nullable = true)
 |-- wind_speed_gust_surface: double (nullable = true)
 |-- _c18: string

In [5]:
sp = sp.select(['2_lat','3_lon','visibility_surface','pressure_maximum_wind'])

In [6]:
vis = sp.rdd.map(lambda x: x[2]).stats()
vis_mean = vis.mean()
vis_stdev = vis.stdev()

wind = sp.rdd.map(lambda x: x[3]).stats()
wind_mean = wind.mean()
wind_stdev = wind.stdev()

In [7]:
def stdevs(e):
    return ( e[0], ((e[1][0]/e[1][2])-vis_mean)/vis_stdev, ((e[1][1]/e[1][2])-wind_mean)/wind_stdev )

# ( geohash ( vis, wind ) ), sort()
# ( geohash ( vis wind ct ) )
# ( geohash ( vis_dev wind_dev ) )

wind_sun_scores = sp.rdd \
    .map(lambda x: ( gh.encode(x[0],x[1]),( x[2],x[3],1 ) )) \
    .reduceByKey(lambda a,b: ( a[0]+b[0],a[1]+b[1],a[2]+b[2] )) \
    .map(lambda x: stdevs(x)) \
    .cache()

In [8]:
wind_farms = wind_sun_scores.map(lambda x: (x[0],x[2])).sortBy(lambda x: x[1], ascending=False).take(20)
solar_farms = wind_sun_scores.map(lambda x: (x[0],x[1])).sortBy(lambda x: x[1], ascending=False).take(20)
wind_solar_farms = wind_sun_scores.map(lambda x: (x[0],(x[1]+x[2])/2)).sortBy(lambda x: x[1], ascending=False).take(20)

In [9]:
m = folium.Map(location=[39.83,-98.58], zoom_start=4)
mc = MarkerCluster().add_to(m)

for wf in wind_farms:
    folium.Marker(location=list(gh.decode(wf[0])), popup=str(wf[1]), icon=folium.Icon(color='lightblue')).add_to(mc)

for sf in solar_farms:
    folium.Marker(location=list(gh.decode(sf[0])), popup=str(sf[1]), icon=folium.Icon(color='orange')).add_to(mc)

for wsf in wind_solar_farms:
    folium.Marker(location=list(gh.decode(wsf[0])), popup=str(wsf[1]), icon=folium.Icon(color='lightgreen')).add_to(mc)

m
# light blue pins are wind farms
# orange pins are solar farms
# light green pins are combo farms

In [10]:
print(timeit.default_timer()-start)

407.23308689997066
